## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
from sympy import KroneckerDelta
kd = KroneckerDelta

x = Content('x')
x1 = Content('x1')
x2 = Content('x2')
y = Content('y')
y1 = Content('y1')
y2 = Content('y2')

D = 2

pi_nf = OutcomeDistribution.StemCell(Symbol("\pi_F(y,y'|x,x')"), D, y1, y2, x1, x2)

# Proliferation   = TransitionClass( [x] -to> [(1,0)] + [(1,0)],         'k_P', g=x[0]*x[1], name="P" )
# Differentiation = TransitionClass( [x] -to> [(1,0)] + [(0,0)],         'k_D', g=x[0]*x[1], name="D" )
# Ageing          = TransitionClass( [x] -to> [x + (0,1)],               'k_a', g=x[0],      name="a" )
# Exit            = TransitionClass( [x] -to> {},                        'k_E', g=1-x[0],    name="E" )
Proliferation   = TransitionClass( [x] -to> [(1,0)] + [(1,0)],         'k_P', g=kd(x[0],1)*x[1], name="P" )
Differentiation = TransitionClass( [x] -to> [(1,0)] + [(0,0)],         'k_D', g=kd(x[0],1)*x[1], name="D" )
Ageing          = TransitionClass( [x] -to> [x + (0,1)],               'k_a', g=kd(x[0],1),      name="a" )
Exit            = TransitionClass( [x] -to> {},                        'k_E', g=kd(x[0],0),      name="E" )
# Exit            = TransitionClass( [x] -to> {},                        'k_E',                    name="E" )
# Proliferation   = TransitionClass( [x] -to> [(1,0)] + [(1,0)],         'k_P', g=x[1], name="P" )
# Differentiation = TransitionClass( [x] -to> [(1,0)] + [(0,0)],         'k_D', g=x[1], name="D" )
# Ageing          = TransitionClass( [x] -to> [x + (0,1)],               'k_a', g=x[0],      name="a" )
# Exit            = TransitionClass( [x] -to> {},                        'k_E', g=1-x[0],    name="E" )

# NegFeedback     = TransitionClass( [x1] + [x2] -to> [y1] + [y2], 'k_NF', g=x1[0]*x2[0], pi=pi_nf, name="NF" )
NegFeedback     = TransitionClass( [x1] + [x2] -to> [y1] + [y2], 'k_NF', g=kd(x1[0],1)*kd(x2[0],1), pi=pi_nf, name="NF" )

transitions = [Proliferation, Differentiation, Ageing, Exit, NegFeedback]
# transitions = [Proliferation, Differentiation, Ageing, Exit]
# transitions = [NegFeedback]
# transitions = [Proliferation, Differentiation, Exit]

display_transition_classes(transitions)

                                                        [x] ---> 2*[(1, 0)],  
                                                    [x] ---> [(0, 0)] + [(1, 0
                                                          [x] ---> [(0, 1) + x
                                                            [x] ---> EmptySet,
[x1] + [x2] ---> [y1] + [y2],  h_NF = \pi_F(y,y'|x,x')*k_NF*(n(x2) - Kronecker

h_P = k_P*n(x)*KroneckerDelta(1, x[0])*x[1]                                   
)],  h_D = k_D*n(x)*KroneckerDelta(1, x[0])*x[1]                              
],  h_a = k_a*n(x)*KroneckerDelta(1, x[0])                                    
  h_E = k_E*n(x)*KroneckerDelta(0, x[0])                                      
Delta(x1, x2))*n(x1)*KroneckerDelta(1, x1[0])*KroneckerDelta(1, x2[0])/(Kronec

                     
                     
                     
                     
kerDelta(x1, x2) + 1)

In [5]:
M = Moment
N = M(0,0)
E = Expectation
kd = KroneckerDelta

moments = [
    # All cells
    N,
    N**2,
    # Stem cells
    M(1,0),
    M(1,0)**2,
    # Differentiated cells
#     (N-M(1,0)),
#     (N-M(1,0))**2,
#     # All age
#     Moment(0,1),
#     Moment(0,1)**2,
#     # Stem cell age
#     M(1,1),
#     M(1,2),
#     M(1,3)
]
display(moments)

# customClosures = [ 
# #                     ( M(0,1)*M(1,1), E(M(1,1)) ), 
# #                     ( N*M(0,1), E(N)*E(M(0,1)) ), 
# #                     ( N*M(1,0), E(N)*E(M(1,0)) ), 
#                  ]
# customSubstitutions= [ 
# #                         ( E(M(0,1))*E(M(1,0))/E(N), E(M(1,1))),
# #                         ( E(M(1,1)), E(M(0,1)) ),
# #                         ( M(1,1), M(0,1) ),
# #                         ( E(M(0,1))*E(M(1,0)**2)/E(N)**2, E(M(0,1))*E(M(1,0))/E(N) ),
# #                         ( M(1,1), M(0,1) ),
#                         ( x[0]**2, x[0]),
#                      ]

customSubstitutions = [
#     ( kd(x[0],0)*x[1], 0 ),
#     ( kd(x1[0],0)*x1[1], 0 ),
#     ( kd(x2[0],0)*x2[1], 0 ),
#     ( kd(y[0],0)*y[1], 0 ),
#     ( kd(y1[0],0)*y1[1], 0 ),
#     ( kd(y2[0],0)*y2[1], 0 ),
    ( M(1,1), M(0,1) ),
#     ( M(1,2), M(0,2) ),
#     ( M(0,1), M(1,1) ),
]
# customClosures = [
#     ( M(1,0)**3, E(M(1,0))*E(M(1,0)**2) ),
# ]

equations = automated_moment_equations(D, transitions, moments, custom_substitutions=customSubstitutions,
# equations = compute_moment_equations(transitions, moments, substitutions=customSubstitutions, 
                                       clna=True)
#                                        clna=[False, True, False, True, False, False])
#                                        clna=False)
display_moment_equations(equations)

⎡                    2                            2      ⎤
⎣Moment(0, 0), Moment (0, 0), Moment(1, 0), Moment (1, 0)⎦

dfM: DeltaM(0, 0)
monomials: [(1, 1, DeltaM(0, 0))]
contentFun: x[0]*x[1]
contentFun_clna: E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: k_P*Moment(0, 1)*E[Moment(1, 0)]/E[Moment(0, 0)]
contentFun: x[0]*x[1]
contentFun_clna: E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: k_D*Moment(0, 1)*E[Moment(1, 0)]/E[Moment(0, 0)]
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: x[0] - 1
contentFun_clna: -1 + E[Moment(1, 0)]/E[Moment(0, 0)]
dfMdt: k_E*(-E[Moment(0, 0)] + E[Moment(1, 0)])*Moment(0, 0)/E[Moment(0, 0)]
contentFun: 0
contentFun_clna: 0
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables={0}
	l_n_Xc: 0
	replaced1: [0]
	replaced2: [0]
dfMdt: 0
dfM: DeltaM(0, 0)**2 + 2*DeltaM(0, 0)*Moment(0, 0)
monomials: [(1, 1, DeltaM(0, 0)**2), (2, Moment(0, 0), DeltaM(0, 0))]
contentFun: x[0]*x[1]
contentFun_clna: E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: k_P*Moment(0, 1)*E[Moment(1, 0)]/E[Moment(0, 0)]
contentFun: x[0]*x[1]
contentFun_clna: E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: 2*k_P*M

dfMdt: k_NF*(-(2*Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + Moment(0, 0))*Moment(1, 0)*E[Moment(1, 0)]**2/E[Moment(0, 0)]**2
dfM: DeltaM(0, 1)*DeltaM(1, 0) + DeltaM(0, 1)*Moment(1, 0) + DeltaM(1, 0)*Moment(0, 1)
monomials: [(1, 1, DeltaM(0, 1)*DeltaM(1, 0)), (1, Moment(1, 0), DeltaM(0, 1)), (1, Moment(0, 1), DeltaM(1, 0))]
contentFun: -x[0]*x[1]**2
contentFun_clna: (-2*E[Moment(0, 0)]*x[1] + E[Moment(0, 1)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
dfMdt: k_P*(Moment(0, 0)*E[Moment(0, 1)] - 2*Moment(0, 1)*E[Moment(0, 0)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
contentFun: -x[0]*x[1]**2
contentFun_clna: (-2*E[Moment(0, 0)]*x[1] + E[Moment(0, 1)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
dfMdt: k_P*(Moment(0, 0)*E[Moment(0, 1)] - 2*Moment(0, 1)*E[Moment(0, 0)])*Moment(1, 0)*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
contentFun: x[0]*x[1]
contentFun_clna: E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: k_P*Moment(0, 1)**2*E[Moment(1, 0)]/E[Moment

dfMdt: k_NF*(-Moment(0, 0)*E[Moment(0, 1)] - Moment(0, 1)*E[Moment(0, 0)] + Moment(0, 1) + E[Moment(0, 0)]*E[Moment(0, 1)])*Moment(0, 0)*E[Moment(1, 0)]**2/(2*E[Moment(0, 0)]**2)
dfM: DeltaM(0, 1)
monomials: [(1, 1, DeltaM(0, 1))]
contentFun: -x[0]*x[1]**2
contentFun_clna: (-2*E[Moment(0, 0)]*x[1] + E[Moment(0, 1)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
dfMdt: k_P*(Moment(0, 0)*E[Moment(0, 1)] - 2*Moment(0, 1)*E[Moment(0, 0)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
contentFun: -x[0]*x[1]**2
contentFun_clna: (-2*E[Moment(0, 0)]*x[1] + E[Moment(0, 1)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
dfMdt: k_D*(Moment(0, 0)*E[Moment(0, 1)] - 2*Moment(0, 1)*E[Moment(0, 0)])*E[Moment(0, 1)]*E[Moment(1, 0)]/E[Moment(0, 0)]**3
contentFun: x[0]
contentFun_clna: E[Moment(1, 0)]/E[Moment(0, 0)]
dfMdt: k_a*Moment(0, 0)*E[Moment(1, 0)]/E[Moment(0, 0)]
contentFun: -(1 - x[0])*x[1]
contentFun_clna: -x[1] + E[Moment(1, 0)]*x[1]/E[Moment(0, 0)]
dfMdt: k_E*(-E[Moment(0, 0)]

dfMdt: k_NF*(-Moment(0, 0)*E[Moment(0, 1)] - Moment(0, 1)*E[Moment(0, 0)] + Moment(0, 1) + E[Moment(0, 0)]*E[Moment(0, 1)])*Moment(1, 0)*E[Moment(1, 0)]**2/(2*E[Moment(0, 0)]**2)
contentFun: -x1[0]*x2[0]
contentFun_clna: -E[Moment(1, 0)]**2/E[Moment(0, 0)]**2
CHECK BOOLS: alpha=[0, 0], beta=[0, 0], boolean_variables={0}
	l_n_Xc: -k_NF*Moment(0, 1)*E[Moment(1, 0)]**2/E[Moment(0, 0)]**2
	replaced1: [-k_NF*(2*(Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + E[Moment(0, 0)]**2)*Moment(0, 1)*E[Moment(1, 0)]**2/(2*E[Moment(0, 0)]**2)]
	replaced2: [-k_NF*Moment(0, 0)*Moment(0, 1)*E[Moment(1, 0)]**2/(2*E[Moment(0, 0)]**2)]
dfMdt: k_NF*(-(2*Moment(0, 0) - E[Moment(0, 0)])*E[Moment(0, 0)] + Moment(0, 0))*Moment(0, 1)*E[Moment(1, 0)]**2/(2*E[Moment(0, 0)]**2)
dfM: DeltaM(0, 0)*DeltaM(1, 0) + DeltaM(0, 0)*Moment(1, 0) + DeltaM(1, 0)*Moment(0, 0)
monomials: [(1, 1, DeltaM(0, 0)*DeltaM(1, 0)), (1, Moment(1, 0), DeltaM(0, 0)), (1, Moment(0, 0), DeltaM(1, 0))]
contentFun: x[0]*x[1]
contentFun_clna: 

...

                                                                              
d                                            2⋅(k_D + k_P)⋅Expectation(N⋅Momen
──(Expectation(Moment(0, 1)⋅Moment(1, 0))) = ─────────────────────────────────
dt                                                                            
                                                                              

                    2                                           ⎛             
t(1, 0))⋅Expectation (Moment(0, 1))⋅Expectation(Moment(1, 0)) + ⎝-2⋅k_E⋅Expect
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                             2
ation(Moment(0, 1)⋅Moment(1, 0)) + k_NF⋅Expectation(Moment(0, 1))⋅Expectation 
──────────────────────────────────────────────────

                                                  ⎛ 2⎞            2           
d                                 k_NF⋅Expectation⎝N ⎠⋅Expectation (Moment(1, 
──(Expectation(N⋅Moment(1, 0))) = ────────────────────────────────────────────
dt                                                                            
                                                                              

      ⎛                                                                    2  
0)) + ⎝-2⋅k_E⋅Expectation(N⋅Moment(1, 0)) + k_NF⋅Expectation(N)⋅Expectation (M
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

            ⎞            2        ⎛                                           
oment(1, 0))⎠⋅Expectation (N) + 2⋅⎝k_D⋅Expectation(Moment(0, 1)⋅Moment(1, 0)) 
──────────────────────────────────────────────────

                                                           ⎛ 2⎞            2  
d                                 2⋅(k_D + k_P)⋅Expectation⎝N ⎠⋅Expectation (M
──(Expectation(N⋅Moment(0, 1))) = ────────────────────────────────────────────
dt                                                                            
                                                                              

                                         ⎛                                    
oment(0, 1))⋅Expectation(Moment(1, 0)) + ⎝-4⋅k_E⋅Expectation(N⋅Moment(0, 1)) +
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                             2
 2⋅k_E⋅Expectation(Moment(0, 1)) + k_NF⋅Expectation(Moment(0, 1))⋅Expectation 
──────────────────────────────────────────────────

                                                                              
d                               (2⋅k_E - k_NF⋅Expectation(Moment(1, 0)))⋅Expec
──(Expectation(Moment(0, 1))) = ──────────────────────────────────────────────
dt                                                                            
                                                                              

                                                                              
tation(N)⋅Expectation(Moment(0, 1))⋅Expectation(Moment(1, 0)) + 2⋅(-k_E⋅Expect
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                               2              
ation(Moment(0, 1)) + kₐ⋅Expectation(Moment(1, 0)))⋅Expectation (N) + (-2⋅k_D⋅
──────────────────────────────────────────────────

                                   ⎛                   ⎛      2      ⎞        
d ⎛           ⎛      2      ⎞⎞   2⋅⎝- 2⋅k_E⋅Expectation⎝Moment (0, 1)⎠ + kₐ⋅Ex
──⎝Expectation⎝Moment (0, 1)⎠⎠ = ─────────────────────────────────────────────
dt                                                                            
                                                                              

                       ⎞            3                                         
pectation(Moment(1, 0))⎠⋅Expectation (N) + (4⋅k_D⋅Expectation(N⋅Moment(0, 1)) 
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                              
+ 2⋅k_D⋅Expectation(Moment(0, 1)) - k_NF⋅Expectation(Moment(1, 0)) + 4⋅k_P⋅Exp
──────────────────────────────────────────────────

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
# python_code = generate_python_code(equations, function_name="stem_cells")
# print(python_code)
# exec(python_code) 

In [7]:
julia_code = generate_julia_code(equations, function_name="StemCells")
print(julia_code)

# evaluate ODEs
function StemCells_ODEs(dM, M, parameters, t)
  c0 = parameters[:kD] # kD
  c2 = parameters[:kE] # kE
  c4 = parameters[:kNF] # kNF
  c3 = parameters[:kP] # kP
  c1 = parameters[:ka] # ka
  # Number of Compartments (N)
  dM[1] = (c0*M[8]*M[3]+c3*M[8]*M[3]-c2*(-M[3]+M[1])*M[1])/M[1]
  # N^2
  dM[2] = (c0*M[8]*M[3]+c3*M[8]*M[3]-c2*(-M[1]+2*M[2]+M[3])*M[1]+2*c0*M[7]*M[3]+2*c2*M[2]*M[3]+2*c3*M[7]*M[3])/M[1]
  # Total Mass of species 1
  dM[3] = 1/2*(c4*M[3]+2*c3*M[8]-c4*M[1]*M[3])*M[3]/M[1]
  # M1^2
  dM[4] = 1/2*(-(c4*M[3]-4*c3*M[5]-2*c3*M[8]+4*c4*M[6]*M[3])*M[1]+2*c4*M[6]*M[3]+c4*M[1]^2*(1+2*M[3])*M[3])*M[3]/M[1]^2
  # M2*M1
  dM[5] = 1/2*(M[1]^3*(-2*c2*M[5]+c4*M[3]^2*M[8])+M[1]^2*(2*c2*M[5]+2*c3*M[9]+2*c1*M[6]+c4*M[3]^2*M[8]+c4*M[8]*M[3]-c4*M[5]*M[3]-2*c4*M[7]*M[3])*M[3]-(2*c3*M[8]^2+c4*M[8]*M[3]-c4*M[7]*M[3]-c4*M[5]*M[3]+4*c0*M[5]*M[8]+4*c3*M[5]*M[8]+c4*M[6]*M[8]*M[3])*M[1]*M[3]+2*M[8]^2*(c0+c3)*M[6]*M[3])/M[1]^3
  # N*M1
  dM[6] = 1/2*(M[1]^2*(-2*c2*M[6]+c4*M[3]^2*M[1]

In [8]:
raise RuntimeError("STOP HERE PLZ!")

RuntimeError: STOP HERE PLZ!